# Abschnitt "structures":
Überschrift: "Datenstrukturen und Algorithmen".


In [ ]:
import sys
import os
import string
import time
import re
import datetime
import codecs
import itertools
import numpy as np
import sympy as sp
import pycurl
import matplotlib.pyplot as plt

from copy import deepcopy
from scipy.special import binom
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from collections import deque

# Ausgabe von Rechnername und Datum:
hostname = os.uname().nodename.split('.')[0].capitalize()
last_started = datetime.date.today()
print(f"Notebook wurde zuletzt gestartet {last_started} am Rechner {hostname}.")

In [ ]:
# OMIT Moodle-Output ausschalten

In [ ]:
# Selbstgebastelte Module:
from mf.lecture import write_code_snippet, write_function_snippet, MF_ROOTPATH
from mf.texutils import make_standalone_pdf, find_occurrences
from mf.binary_trees import OTHER_SIDE, BinaryTreeNode, BinaryTree, get_list_of_nodes, draw_binary_tree, simple_key_in_circle, adhoc_for_kd_tree, AVLTreeNode, AVLTree, key_n_length_in_circle
from mf.geo import npp, get_line_equation, get_solution_xy, get_norm, get_normalvector, get_angle, get_normalized, get_normalized_normal
from mf.picture import PstricksPicture
from mf.decorator import ps_decorator_factory

# Root folder:
ROOTPATH = '/Users/mfulmek/ucloud/books/dmti/'
TESTPATH = '/Users/mfulmek/ucloud/books/test/'

# Decorator:
ps_dmti = ps_decorator_factory(rootpath=ROOTPATH)
# Für Testzwecke:
ps_test = ps_decorator_factory(rootpath=TESTPATH)

# Name und Nummer des Files für das aktuelle Kapitel:
NUMBER = 6
SECTION = 'structures'
SOURCE = os.path.join(ROOTPATH, "snippets", f'{SECTION}_snippets.ipynb')

# Keyword-Arguments für die Funktionsaufrufe:
COMMON = {
    'rootpath' : ROOTPATH,
    'the_source' : SOURCE,
    'only_python' : False
}

ONLY_PYTHON = {
    'rootpath' : MF_ROOTPATH,
    'the_source' : SOURCE,
    'only_python' : True
}

# Ausgabe von Kapitel:
print(f"Snippets für Kapitel 6: Datenstrukturen und Algorithmen.")

### Adhoc-Funktion, für ein Übungsbeispiel

In [ ]:
def quicksort(objekte):
    """Diese Funktion gibt eine sehr knappe Implementation des
    Quicksort-Algorithmus: Das Argument objekte sollte eine Liste
    (oder ein anderer Iterator) von Objekten sein, die paarweise
    vergleichbar sind (für die also die Operatoren '>' und '<='
    definiert sind), der Rückgabewert ist die sortierte Liste der
    Objekte."""
    # Illustration: Eine Liste oder ein Tupel ist
    # zwar an sich kein Wahrheitswert ("True" oder "False"),
    # aber: Eine Liste ist im Zusammenhang einer if-Bedingung
    # "falsy", wenn sie leer ist, sonst "truthy".
    if not objekte:
        # Für eine leere Liste ist natürlich nichts weiter zu tun
        return []
    # Illustration von _Unpacking_: Das erste Element von
    # objekte wird der variablen trenner zugewiesen, die restlichen
    # Elemente bilden die _neue_ Liste objekte:
    trenner, *objekte = objekte
    # Die restlichen Elemente werden in zwei Teile geteilt, je
    # nachdem ob sie kleinergleich oder größer
    kleinere = [o for o in objekte if o <= trenner]
    groessere = [o for o in objekte if o > trenner]
    # Rekursiver Aufruf der Funktion:
    return quicksort(kleinere) + [trenner] + quicksort(groessere)

In [ ]:
write_function_snippet(quicksort,
    **COMMON,
    the_caption=r'Rekursive Quicksort--Implementierung.',
    preamble = ''
)

In [ ]:
# MOODLE Moodle-Output wieder einschalten

## Binary Search:

Finde die richtige Position $p$, an der ein Element $x$ in eine (aufsteigend) geordnete Liste eingefügt werden müßte, damit die Ordnung erhalten bleibt. Einfügen bedeutet hier: Verschiebe alle Elemente ab $p$ nach rechts, und füge an der dadurch freigewordenen Stelle das Element $x$ ein. Bedingungsloses Einfügen in diesem Sinne kann Doubletten erzeugen, also mehrfache Vorkommnisse desselben Elements - wenn man nur Listen von verschiedenen Elementen möchte, muß man das Einfügen an die Bedingung knüpfen, daß $x$ nicht mit dem Element in Position $p$ übereinstimmt.

In [ ]:
def binary_search(s_list, x):
    """Find the position p (0 <= p <= len(s_list) where element x
    should be _inserted_ in list s_list, which should be sorted
    in _increasing_ order. (Insertion might mean: _Replace_ an
    element which is already present in s_list.)"""
    
    # Length of the list:
    n = len(s_list)
    
    # Simple case:
    if s_list[0] >= x:
        return 0
    if x > s_list[-1]:
        return n
    
    # Otherwise: s_list[0] < x <= s_list[n-1]
    left = 0
    right = n-1
    
    # So we have s_list[left] < x <= s_list[right];
    # we shall _maintain_ these inequalities during
    # the following loop:
    while (right - left) > 1:
        # mid = largest INTEGER <= (right + left) / 2:
        # Symbol "//" means "integer division"; not
        # to be confused with "normal" division "/".
        mid = (right + left) // 2 
        if x <= s_list[mid]:
            right = mid
        else:
            left = mid
    
    return right

In [ ]:
write_function_snippet(binary_search,
    **COMMON,
    the_caption=r'Binary Search: Finde die richtige Position $p$, an der ein Element $x$ in eine (aufsteigend) geordnete Liste eingefügt werden müßte, damit die Ordnung erhalten bleibt. Einfügen bedeutet hier: Verschiebe alle Elemente ab $p$ nach rechts, und füge an der dadurch freigewordenen Stelle das Element $x$ ein. Bedingungsloses Einfügen in diesem Sinne kann Doubletten erzeugen, also mehrfache Vorkommnisse desselben Elements --- wenn man nur Listen von verschiedenen Elementen möchte, muß man das Einfügen an die Bedingung knüpfen, daß $x$ nicht mit dem Element in Position $p$ übereinstimmt.',
    preamble = ''
)

## Knoten für Doubly Linked Lists (doppelt verkettete Listen).


In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
# Ein "Hilfs-Dictionary" für eine "Spiegelung"
# (Vertauschung von links und rechts), die sich hier
# als nützlich erweist:
mirror = {'left' : 'right', 'right' : 'left'}

class DLLNode:
    """A node to be used in a doubly linked list"""
    def __init__(self, data):
        # Store the data
        self.contents = data
        # Initially, the "pointers" to the left/right neighbours
        # are None
        self.neighbour = {'left' : None, 'right' : None}

    def __str__(self):
        """String representation of the node"""
        return f'{self.contents} {self.neighbour}'

    # "Hineinquetschen" eines neuen Knotens, links oder rechts
    # von self:
    def squeeze_in(self, what, where):
        """Insert a new node (containing "what") to side "where".
        """
        # Make the new node containing "what":
        new_node =  DLLNode(what)
        # Self's neighbour on side "where" becomes new node`s neighbour:
        new_node.neighbour[where] = self.neighbour[where]
        if self.neighbour[where]:
            self.neighbour[where].neighbour[mirror[where]] = new_node
        # Make this new node self's neighbour on side "where":
        self.neighbour[where] = new_node
        new_node.neighbour[mirror[where]] = self
        return new_node

    # Entfernen von self aus seiner "Nachbarschaft":
    def detach(self):
        """Remove self."""
        # Nachbarn von self:
        left, right = [self.neighbour[where] for where in ["left", "right"]]
        if left:
            left.neighbour["right"] = right
        if right:
            right.neighbour["left"] = left
        return self

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'DLLNode',
    **COMMON,
    the_caption = r'Knoten für Doubly Linked Lists (doppelt verkettete Listen).',
    preamble = ''
)

## Doubly Linked List:

Einr doppelt verkettete Liste besteht im wesentlichen aus Zeigern (englisch: Pointer) auf den ersten und letzten Knoten der Liste.

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
class DoublyLinkedList:
    """Implementation of a doubly linked list."""
    def __init__(self):
        # Initially, the list is empty: Pointers to
        # left and right ends are None.
        self.end = {'left' : None, 'right' : None}
        self.length = 0

    def apply(self, the_func, *additional_arguments):
        """Traverse the list from left to right and
        apply function the_func to the data stored;
        return list of results."""
        # Beachte die Möglichkeit, der Funktion the_func
        # eine _variable_ Zahl zusätzlicher Argumente
        # zu übergeben (0 ist auch möglich!)
        results = []
        pointer = self.end['left']
        while pointer:
            results+= [the_func(pointer, *additional_arguments)]
            pointer = pointer.neighbour['right']
        return results

    # Beispiel, wie "apply" verwendet werden kann:
    def list_of_nodes(self):
        """Return the doubly linked list as a "normal" python
        list of nodes"""
        return self.apply(lambda x : x)

    # Noch ein Beispiel:
    def __str__(self):
        """String representation of the list"""
        data_list = self.apply(lambda x : f'{x.contents}')
        return f"DLL of length ({self.length}): (" + ", ".join(data_list)+")"

    # Anfügen bzw. Entfernen von Listen heißt "traditionell"
    # "push" bzw. "pop": Bei einer doppelt verketteten Liste
    # können wir links oder rechts anfügen bzw. entfernen.
    def push(self, data, where='right'):
        """Push (i.e., append) new data at list's end "where"."""
        if self.end[where]:
            new_end = self.end[where].squeeze_in(data, where)
        else:
            new_end = DLLNode(data)
            self.end[mirror[where]] = new_end

        self.length+= 1
        self.end[where] = new_end

    def pop(self, where='right'):
        """Remove node and return its data at list's end "where"."""
        if self.end[where]:
            # Funktion "detach" gibt den entfernten Knoten zurück:
            return self.end[where].detach().contents
            self.length-= 1
        # Implicit else:
        print('list is empty!!!')
        return None

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'DoublyLinkedList',
    **COMMON,
    the_caption = r"""Doubly Linked List: Im wesentlichen Zeiger (englisch: Pointer)
    auf ersten und letzten Knoten der doppelt verketteten Liste.""",
    preamble = ''
)

## Beispiel zur Verwendung von ```deque``` (Modul ```collections```)

Für das _breadth-first_-Durchlaufen eines Wurzelbaums ist eine Liste nützlich, bei der wir "hinten und vorne" anfügen oder entfernen können: Unsere ```DoublyLinkedList``` bietet diese Funktionalität, aber auch das Python-Objekt ```deque``` aus dem Modul ```collections```.

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!

# Leeres deque-Objekt:
test = deque()
# "Befüllen"  von rechts (ginge auch von links, mit appendleft):
for i in range(3):
    test.append(i)
    print(test)
# "Leeren" von links (ginge auch von rechts, mit pop):
for i in range(3):
    c = test.popleft()
    print(f'pop: {c}, Rest: {test}')

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'deque',
    **COMMON,
    the_caption = r'Beispiel zur Verwendung von \pythoncode{deque} (Module \pythoncode{collections}).',
    preamble = '# ??? from collections import deque'
)

## Binary search trees

Binary search funktioniert gut in einer konstanten sortierten List. Wir brauchen aber häufig eine "Liste von Objekten", die zwar auch _geordnet_ ist, die aber regelmäßig verändert wird (durch Einfügen oder Löschen): Dafür sind _binäre Bäume_ genau das Richtige.

Hier ein "Muster" für eine Klasse, die einen Knoten in einem Binary Search Tree darstellt: Die Implementierung (ohne die ganzen Kommentare nur wenige Zeilen) sind Gegenstand eines Übungsbeispiels.

Beachten Sie, daß ein _einzelner_ Knoten immer auch als ein _Baum_ betrachtet werden kann, dessen _Wurzel_ er ist!

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!

# Kleiner Trick: "Übersetzung andere Seite" (also "links" <-> "rechts"):
OTHER_SIDE = {
    "left" : "right",
    "right" : "left"
}

# Eine Klasse für einen Knoten - enthält (fast) alles, was für einen
# binary tree nötig ist:
class BinaryTreeNode:
    """A binary tree _basically_ is a _root_ (together with
    all its subtrees).
    
    A node in a binary tree may be viewed as the
    root of the subtrees pending from it and thus as
    a binary tree itself.
    
    In this sense there is little difference
    between binary trees and nodes of binary trees."""
    def __init__(self, key, data = None, parent = None, node_type = "root"):
        """Initialize:"""
        # "Pointer" (sort of...) to parent node
        self.parent = parent
        # Is this node the "global" root, or a left or right child
        # of its parent?
        self.node_type = node_type
        # Informations on left/right children:
        self.child = {"left" : None, "right" : None}
        # And, of course: Store the key ...
        self.key = key
        # ... and the data _as a dict_: Maybe we want to add other data
        # later for the _same_ key!
        self.contents = dict() if data is None else deepcopy(data)

    def __str__(self):
        """Return string representation:"""
        return f'{self.node_type} ({self.key}) <{self.contents}>.'
    
    # Auxiliary functions:
    def is_leaf(self):
        """Return True if self is, in fact, a leaf of the tree
        it belongs to:"""
        return (self.child["left"] is None) and (self.child["right"] is None)
    
    # Append child:
    def append(self, side, child):
        """Append BinaryTreeNode as a (left or right) child:"""
        self.child[side] = child
        if child is not None:
            child.node_type=side
            child.parent = self

    # Search depth-first for key in subtree rooted at self:
    def df_search_subtree(self, key):
        """Depth-First-Suchen nach Knoten key."""
        if self.key == key:
            # Found node "key"!
            return self
        
        # Implicit else:
        # Search left & right subtree by (somewhat recursive) function calls
        # for the child-nodes:

        # Fügen Sie hier bitte geeignete Codezeilen ein!

        # Implicit else:
        # Key not found yet, and no more subtree to search!
        return None

    # Traverse the tree rooted at self "depth-first" and apply a function
    # which takes (node, level_of_node, account) as arguments:
    def df_traverse(self, the_func, level=0, account=None):
        """Durchlaufen des gesamten Baumes, depth-first, wobei
        für jeden Knoten eine Funktion "the_func" aufgerufen wird,
        die auch das aktuelle level (Niveau) und ein (optionales)
        Dictionary "account" (in dem eventuell Informationen
        gespeichert werden; z.B. eine Liste der Blätter) als
        Argumente übernimmt."""
        level = level
        the_func(self, level, account)
        # Traverse left & right subtree by recursive function call:

        # Fügen Sie hier bitte geeignete Codezeilen ein!

        
    # Traverse the tree rooted at self "breadth-first" and apply
    # a function which takes (node, level_of_node, number_of_node)
    # and an optional "account" (in most cases: a dictionary to be
    # updated by the nodes) as arguments:
    def bf_traverse(self, the_func, account=None):
        """Durchlaufen des gesamten Baumes, breadth-first, wobei
        für jeden Knoten eine Funktion "the_func" aufgerufen wird,
        die auch das aktuelle level (Niveau), die Nummer des Knotens
        "in seinem Niveau" und ein (optionales) Dictionary
        "account" (in dem eventuell Informationen gespeichert
        werden; z.B. eine Liste der Blätter) als Argumente
        übernimmt."""
        # Use a queue for storing the nodes (in their order:
        # "level by level, from left to right":
        queue = deque()
        # Also store the level (distance from root) and the
        # number (in its level, from left to right) of the node.
        
        # Initially, append (root,level=0) to the right of the queue,
        # which contains the list of nodes yet to be processed:
        queue.append((self,0))
        # Traverse the tree:
        count = 0
        old_level = -1 # If new level is reached, then reset count!
        while queue:
            # Pop first node (from the left) ...
            node,level = queue.popleft()
            # Update count of node:

            # Fügen Sie hier bitte geeignete Codezeilen ein!

            # Don't forget to apply the_func:
            count+=1
            the_func(node,level,count,account=account)

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'BinaryTreeNode',
    **COMMON,
    the_caption = r'Knoten in einem Binärbaum: Im Wesentlichen zwei Zeiger (englisch: Pointer) auf linken und rechten Nachfolger ("child"), einer auf Vorgänger ("parent").',
    preamble = 'from copy import deepcopy\nfrom collections import deque'
)

Wie gesagt: Ein Wurzelbaum ist "technisch gesehen" im wesentlichen die Wurzel (und die daran hängenden Teilbäume).

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
class BinaryTree:
    """Basically, a BinaryTree is defined by its root
    (and all the subtrees pending from it)."""
    def __init__(self, key, data = None, parent = None, node_type = "root"):
        """Initialize: Either with existing BinaryTreeNode,
        or with key."""
        if isinstance(key, BinaryTreeNode):
            self.root = key
        else:
            self.root = BinaryTreeNode(key, data=data)
    
    def __str__(self):
        return f'BinaryTree (root: {self.root})'

    # Search depth-first for key:
    def df_search_subtree(self, key):
        return self.root.df_search_subtree(key)
    
    # Traverse the tree depth-first and apply a function which
    # takes (node, level_of_node) as arguments
    def df_traverse(self, the_func, level, account):
        self.root.df_traverse(the_func, level, account)
        
    # Traverse the tree (starting at root) breadth-first and apply
    # a function which takes (node, level_of_node, number_of_node)
    # and an optional "account" (in most cases: a dictionary to be
    # updated by the nodes) as arguments:
    def bf_traverse(self, the_func, account):
        self.root.bf_traverse(self, the_func, account=account)

    # Der Baum muss verändert werden können, durch Anhängen oder
    # Löschen von Blättern:
    def append_leaf(self, child, parent_key, side="left"):
        # Find the node parent_key:
        if (parent_node := self.df_search_subtree(parent_key)) is None:
            print(f"Key {parent_key} not found!")
            return
        # Implicit else:

        # Fügen Sie hier bitte geeignete Codezeilen ein!
    
    def delete_leaf(self, key):
        """Find and delete leaf with key."""
        # Find the node key:
        if (node := self.df_search_subtree(key)) is None:
            print(f"No node {key} in binary tree!")
            return
        # Implicit else:
        
        # Fügen Sie hier bitte geeignete Codezeilen ein!

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'BinaryTree',
    **COMMON,
    the_caption = r'Binärbaum: Im Wesentlichen die Wurzel.',
    preamble = ''
)

## Orthogonal range search, eindimensional.


### Hilfsfunktion:

Finde den _ersten_ Knoten, der _innerhalb_ des gegebenen Intervalls liegt  - dort _gabelt_ sich der Weg für eindimensionales orthogonal search.

In [ ]:
def find_split_node(binary_tree, x_min, x_max):
    # Beginne bei der Wurzel von binary_tree:
    v = binary_tree.root
    # Solange v kein Blatt ist und das Intervall (x_min, x_max)
    # "entweder links oder rechts" vom aktuellen Knoten v liegt ...
    while not v.is_leaf() and (x_max <= v.key or x_min > v.key):
        # ... setze mit "richtigem Teilbaum" fort:
        if x_max <= v.key:
            v = v.left
        else:
            v = v.right
    # Ok: v ist entweder ein Blatt, oder x_min <= v.key < x_max
    return v

In [ ]:
write_function_snippet(find_split_node,
    **COMMON,
    the_caption=r'Hilfsfunktion: Finde den ``ersten'' Knoten, der ``innerhalb'' des gegebenen Intervalls liegt  --- dort ``gabelt'' sich der Weg für eindimensionales orthogonal search.',
    preamble = ''
)

In [ ]:
def onedim_range_query(binary_tree, x_min, x_max):
    # Hilfsfunktion: 
    def is_in_range(x):
        return x_min <= x.key and x.key < x_max
    # Liste der gefundenen Knoten:
    global report
    report = []
    # Hilfsfunktion: Gib alle Blätter von Teilbaum in Liste report
    def update_report(node,level,count):
        global report
        if node.is_leaf():
            report+= [node.key]
    
    # Finde den ersten Knoten, der innerhalb des Intervalls liegt:
    split_node = find_split_node(binary_tree, x_min, x_max)
    
    # Es kann natürlich sein, daß dieser Knoten ein Blatt ist:
    if split_node.is_leaf():
        # Wenn dieses Blatt im gesuchten Bereich liegt, muß
        # im Rückgabewert berücksichtigt werden:
        if is_in_range(split_node.key):
            report+=[split_node.key]
            return report
    
    # Implicit else:
    
    # Beginne Weg mit Kante zum linken Teilbaum:
    v = split_node.left
    while not v.is_leaf():
        if x_min <= v.key:
            v.right.bf_transversal(update_report)
            v = v.left
        else:
            v = v.right
    # v ist nun ein Blatt: Müssen wir es berücksichtigen?
    if is_in_range(v):
        report+=[v.key]

    # Beginne Weg mit Kante zum rechten Teilbaum:
    v = split_node.right
    while not v.is_leaf():
        if x_max > v.key:
            v.left.bf_transversal(update_report)
            v = v.right
        else:
            v = v.left
    # v ist nun ein Blatt: Müssen wir es berücksichtigen?
    if is_in_range(v):
        report+=[v.key]
    
    return sorted(report)

In [ ]:
write_function_snippet(onedim_range_query,
    **COMMON,
    the_caption=r'Orthogonal range search, eindimensional.',
    preamble = ''
)

## Zerlegung einer Punktwolke entlang einer Koordinatenachse.


In [ ]:
def split_cloud_of_points(cloud, subset, axis):
    """Gegeben sei eine "Punktwolke" von m d-dimensionalen Punkten
    in Form einer d x m Matrix (numpy-array) cloud, und eine
    Teilmenge dieser "Punktwolke" in Form eines eindimensionalen
    numpy-arrays (dtype=boolean) subset, das die (Spalten-)Indizes der
    Punkte der Teilmenge subset codiert (im Sinne einer charakteristischen
    Funktion der Teilmenge): Die Funktion liefert eine
    Zerlegung der Teilmenge gemäß dem _Median_ der Projektion
    auf die Koordinaten-Achse axis.
    """ 

    # Projektion: Koordinaten-Achse axis; das bool-Array subset
    # "wählt daraus die Spalten aus, die zu subset gehören":
    coords = cloud[axis][subset]
    
    # coords kann im allgemeinen "Doubletten" (also gleiche Elemente)
    # enthalten: Die numpy-Funktion "unique" returns the sorted unique
    # elements of an array.
    # Hier wird also ("im Hintergrund") sortiert - O(n log(n)), und es
    # werden "Doubletten" entfernt:
    unique_sorted_coords = np.unique(coords)
    
    nof_unique_coords = len(unique_sorted_coords)
    if nof_unique_coords <= 1:
        # Hier gibt's nichts mehr zu "zerschneiden":
        return unique_sorted_coords[0], subset, np.zeros(
            cloud.shape[1],
            dtype=int
        )
    
    # Implicit else:
    
    # Bestimme den Median der Koordinaten (ohne Wiederholungen!):
    median = unique_sorted_coords[(nof_unique_coords+1)//2-1]
    
    # Auch das folgende wird mit numpy sehr einfach:
    # "Im Hintergrund" wird das ganze array durchlaufen - O(n) -;
    # wir erhalten damit die charakteristische Funktion aller
    # Spalten, deren axis-Koordinate <= median ist:
    boolean_array_indicating_left_block = (cloud[axis] <= median)
    
    # Erzeuge zwei neue Blöcke (als _Kopien_ der entsprechenden Zerlegung):
    # numpy logical_and ist eine _koordinatenweise_ UND-Verknüpfung; für
    # die charakteristischen Funktionen von Teilmengen entspricht diese
    # logische Operation der Durchschnittsbildung:
    left_or_lower_block = np.copy(
        np.logical_and(
            subset,
            boolean_array_indicating_left_block
        )
    )
    # numpy logical_not "flippt" jede bool-Eintragung: True->False, und
    # vice versa; für die charakteristischen Funktionen von Teilmengen
    # entspricht diese logische Operaton der Komplementbildung:
    right_or_upper_block = np.copy(
        np.logical_and(
            subset,
            np.logical_not(boolean_array_indicating_left_block)
        )
    )
    
    # Gib Median und Blöcke zurück:
    return median, left_or_lower_block, right_or_upper_block

In [ ]:
write_function_snippet(split_cloud_of_points,
    **COMMON,
    the_caption=r'Zerlegung einer Punktwolke entlang einer Koordinatenachse.',
    preamble = ''
)

## Konstruktion des Kd-(Teil)Baums, der einer Teilmenge von Punkten der Punktwolke entspricht.

### Hilfsfunktion (die alles Wesentliche enthält)

In [ ]:
def kd_subtree(cloud, subset, axis=0):
    """Erzeuge den (Teil-)Baum, der sich aus der sukzessiven
    Zerlegung der Teilmenge subset der Punktwolke cloud ergibt;
    gib die Wurzel dieses Teilbaums zurück."""
    dimension, nof_points = cloud.shape
    
    # Create a new node:
    node = BinaryTreeNode()
    
    # Determine cardinality of subset:
    columns = np.nonzero(subset)[0]
    
    if len(columns) == 0:
        return None
    
    if len(columns) == 1:
        # This is a leaf! Store (a copy of) the (single) point
        # in subset as data (i.e.: Remember column with Index
        # columns[0]) ...
        j = columns[0]
        node.contents = [j,np.copy(cloud[:,j])]
        # ... and the axis-coordinate as key ...
        node.key = (node.contents[1][axis],axis)
        # ... for this node and return it:
        return node

    # Implicit else:
    
    # Split subset of cloud along axis:
    median, left_block, right_block = split_cloud_of_points(
        cloud,
        subset,
        axis
    )
    # Store median and axis as key for this node ...
    node.key = (median,axis)
    
    # ... and continue recursively:
    axis = (axis + 1) % dimension
    # node.append_left(kd_subtree(cloud, subset=left_block, axis=axis))
    node.left = kd_subtree(cloud, subset=left_block, axis=axis)
    if node.left is not None:
        node.left.node_type='left'
        node.left.predecessor = node
    # node.append_right(kd_subtree(cloud, subset=right_block, axis=axis))
    node.right = kd_subtree(cloud, subset=right_block, axis=axis)
    if node.right is not None:
        node.right.node_type='right'
        node.right.predecessor=node
    
    # Finally, return the node (=root of constructed subtree):
    return node

In [ ]:
write_function_snippet(kd_subtree,
    **COMMON,
    the_caption=r'Konstruktion des Kd--(Teil)Baums, der einer Teilmenge von Punkten der Punktwolke entspricht.',
    preamble = ''
)

### Aufruf der Hilfsfunktion zur Konstruktion des Kd-Baums.

In [ ]:
def kd_tree(cloud):
    # cloud.shape[1] = Anzahl der Spalten, also gleich
    # Anzahl der Punkte: Wir betrachten hier die (unechte)
    # Teilmenge _aller_ Punkte.
    subset = np.ones(cloud.shape[1], dtype=bool)
    # Die Wurzel des so konstruierten Teilbaums wird
    # die Wurzel eines BinaryTree, den wir als Resultat
    # zurückgeben.
    return BinaryTree(kd_subtree(cloud, subset))

In [ ]:
write_function_snippet(kd_tree,
    **COMMON,
    the_caption=r'Konstruktion des Kd--Baums, der der Menge von Punkten einer Punktwolke entspricht.',
    preamble = ''
)

In [ ]:
# OMIT - das wurde in binary_trees.py übernommen!!!

## Eine Graphik zum Thema kd-Trees

In [ ]:
CELLNUMBER = len(_ih) - 1
# Kleiner Trick: "Übersetzung andere Seite" (also "links" <-> "rechts"):
OTHER_SIDE = {
    "left" : "right",
    "right" : "left"
}

# Eine Klasse für einen Knoten - enthält (fast) alles, was für einen
# binary tree nötig ist:
class BinaryTreeNode:
    """A binary tree _basically_ is a _root_ (together with
    all its subtrees).

    A node in a binary tree may be viewed as the
    root of the subtrees pending from it and thus as
    a binary tree itself.

    In this sense there is little difference
    between binary trees and nodes of binary trees."""
    def __init__(self, key, data = None, parent = None, node_type = "root"):
        """Initialize:"""
        # "Pointer" (sort of...) to parent node
        self.parent = parent
        # Is this node the "global" root, or a left or right child
        # of its parent?
        self.node_type = node_type
        # Informations on left/right children:
        self.child = {"left" : None, "right" : None}
        # And, of course: Store the key ...
        self.key = key
        self.contents = dict() # dict to be filled with useful information!
        # ... and the data _as a list_: Maybe we want to add other data
        # later for the _same_ key!
        if data is not None:
            self.contents["data"] = [deepcopy(data)]

    def __str__(self):
        """Return string representation:"""
        return f'node: type = {self.node_type}, key = {self.key}, data = {self.contents}.'

    # Auxiliary functions:
    def is_leaf(self):
        """Return True if self is, in fact, a leaf of the tree
        it belongs to:"""
        return (self.child["left"] == self.child["right"]) and (self.child["left"] is None)

    def get_sibling(self):
        if self.parent is None:
            return None
        # Implicit else:
        return self.parent.child[OTHER_SIDE[self.node_type]]

    # Append child:
    def append(self, side, child):
        """Append BinaryTreeNode as a (left or right) child:"""
        self.child[side] = child
        if child is not None:
            child.node_type=side
            child.parent = self

    # Search depth-first for key in subtree rooted at self:
    def df_search_subtree(self, key):
        """Depth-First-Suchen nach Knoten key."""
        if self.key == key:
            return self
        # Implicit else:
        # Search left subtree by recursive function call:
        if (node := self.child["left"]) is not None:
            result = node.df_search_subtree(key)
            if result is not None:
                return result
        # Implicit else:
        # Search right subtree by recursive function call:
        if (node := self.child["right"]) is not None:
            return node.df_search_subtree(key)
        # Implicit else:
        # Key not found yet, and no more subtree to search!
        return None

    # Traverse the tree rooted at self "depth-first" and apply
    # takes (node, level_of_node) as arguments:
    def df_traverse(self, the_func, level=0, account=None):
        """Durchlaufen des gesamten Baumes: Depth-first."""
        # print("BTN.df_traverse: ",account)
        level = level
        the_func(self, level, account)
        # Search left subtree by recursive function call:
        if (node := self.child["left"]) is not None:
            node.df_traverse(the_func, level+1, account)
         # Search right subtree by recursive function call:
        if (node := self.child["right"]) is not None:
            node.df_traverse(the_func, level+1, account)

    # Traverse the tree rooted at self "breadth-first" and apply
    # a function which takes (node, level_of_node, number_of_node)
    # and an optional "account" (in most cases: a dictionary to be
    # updated by the nodes) as arguments:
    def bf_traverse(self, the_func, account=None):
        """Durchlaufen des gesamten Baumes: Breadth-first."""
        # We use a queue for storing the nodes (in their order:
        # "level by level, from left to right":
        queue = deque()
        # Here, we also store the level (distance from root)
        # of the node.

        # Initially, append (root,level=0) to the right of the queue,
        # which contains the list of nodes yet to be processed:
        queue.append((self,0))
        # Traverse the tree:
        count = 0
        old_level = -1
        while queue:
            # Pop first node (from the left) ...
            node,level = queue.popleft()
            # Update count of node:
            if level > old_level:
                # Reset to zero if new level is reached:
                count=0
                old_level=level
            else:
                count+=1
            # ... and append its successors (if any) to the right
            # of the queue:
            if (child := node.child["left"]):
                queue.append((child,level+1))
            if (child := node.child["right"]):
                queue.append((child,level+1))
            # Apply the_func:
            count+=1
            the_func(node,level,count,account=account)


# Ein binary tree ist "technisch gesehen" im wesentlichen seine Wurzel (und
# natürlich die daran hängenden Teilbäume).
class BinaryTree:
    """Basically, a BinaryTree is defined by its root
    (and all the subtrees pending from it)."""
    def __init__(self, key, data=None):
        """Initialize: Either with existing BinaryTreeNode,
        or with key."""
        if isinstance(key, BinaryTreeNode):
            self.root = key
        else:
            self.root = BinaryTreeNode(key, data=data)

    def __str__(self):
        return f'BinaryTree (root: {self.root})'

    # Search depth-first for key:
    def df_search_subtree(self, key):
        return self.root.df_search_subtree(key)

    # Traverse the tree depth-first and apply a function which
    # takes (node, level_of_node) as arguments
    def df_traverse(self, the_func, level, account):
        self.root.df_traverse(the_func, level, account)

    # Traverse the tree (starting at root) breadth-first and apply
    # a function which takes (node, level_of_node, number_of_node)
    # and an optional "account" (in most cases: a dictionary to be
    # updated by the nodes) as arguments:
    def bf_traverse(self, the_func, account):
        self.root.bf_traverse(the_func, account)

    def append_leaf(self, child, parent_key, side="left"):
        # Find the node parent_key:
        if (parent_node := self.df_search_subtree(parent_key)) is None:
            print(f"Key {parent_key} not found!")
            return
        # Implict else:
        if parent_node.child[side] is not None:
            print(f"Node {parent_key} already has {side} child!")
            return
        # Implicit else:
        parent_node.append(side, child)

    def delete_leaf(self, key):
        """Find and delete leaf with key."""
        # Find the node key:
        if (node := self.df_search_subtree(key)) is None:
            print(f"No node {key} in binary tree!")
            return
        # Implicit else:
        if node.is_leaf():
            node.parent.child[node.node_type] = None
        else:
            print(f'Node {key} is no leaf!')
        return

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'BinaryTreeNode',
    **ONLY_PYTHON,
    the_caption = r'BinaryTree(Node).',
    preamble = 'from copy import deepcopy\nfrom collections import deque'
)

In [ ]:
# Graphik "kd_search2" verbessert:
kdt = BinaryTree("v_{22.5}")
kdt.append_leaf(BinaryTreeNode("h_{4.5}"), "v_{22.5}","left")
kdt.append_leaf(BinaryTreeNode("h_{9.5}"), "v_{22.5}","right")

kdt.append_leaf(BinaryTreeNode("v_{18.5}"), "h_{4.5}","left")
kdt.append_leaf(BinaryTreeNode("v_{20.5}"), "h_{4.5}","right")
kdt.append_leaf(BinaryTreeNode("v_{25.0}"), "h_{9.5}","left")
kdt.append_leaf(BinaryTreeNode("v_{25.5}"), "h_{9.5}","right")

kdt.append_leaf(BinaryTreeNode("h_{2.4}"), "v_{18.5}","left")
kdt.append_leaf(BinaryTreeNode("h_{2.6}"), "v_{18.5}","right")
kdt.append_leaf(BinaryTreeNode("h_{7.5}"), "v_{20.5}","left")
kdt.append_leaf(BinaryTreeNode("h_{6.0}"), "v_{20.5}","right")
kdt.append_leaf(BinaryTreeNode("13"), "v_{25.0}","left")
kdt.append_leaf(BinaryTreeNode("17"), "v_{25.0}","right")
kdt.append_leaf(BinaryTreeNode("h_{11.0}"), "v_{25.5}","left")
kdt.append_leaf(BinaryTreeNode("16"), "v_{25.5}","right")

kdt.append_leaf(BinaryTreeNode("h_{1.5}"), "h_{2.4}","left")
kdt.append_leaf(BinaryTreeNode("h_{3.5}"), "h_{2.4}","right")
kdt.append_leaf(BinaryTreeNode("4"), "h_{2.6}","left")
kdt.append_leaf(BinaryTreeNode("5"), "h_{2.6}","right")
kdt.append_leaf(BinaryTreeNode("h_{6.5}"), "h_{7.5}","left")
kdt.append_leaf(BinaryTreeNode("h_{8.5}"), "h_{7.5}","right")
kdt.append_leaf(BinaryTreeNode("v_{21.5}"), "h_{6.0}","left")
kdt.append_leaf(BinaryTreeNode("10"), "h_{6.0}","right")
kdt.append_leaf(BinaryTreeNode("14"), "h_{11.0}","left")
kdt.append_leaf(BinaryTreeNode("15"), "h_{11.0}","right")

kdt.append_leaf(BinaryTreeNode("0"), "h_{1.5}","left")
kdt.append_leaf(BinaryTreeNode("1"), "h_{1.5}","right")
kdt.append_leaf(BinaryTreeNode("2"), "h_{3.5}","left")
kdt.append_leaf(BinaryTreeNode("3"), "h_{3.5}","right")
# kdt.append_leaf(BinaryTreeNode("10"), "h_{6.0}","left")
# kdt.append_leaf(BinaryTreeNode("v_{21.5}"), "h_{6.0}","right")
kdt.append_leaf(BinaryTreeNode("7"), "h_{6.5}","left")
kdt.append_leaf(BinaryTreeNode("6"), "h_{6.5}","right")
kdt.append_leaf(BinaryTreeNode("8"), "h_{8.5}","left")
kdt.append_leaf(BinaryTreeNode("9"), "h_{8.5}","right")

kdt.append_leaf(BinaryTreeNode("11"), "v_{21.5}","left")
kdt.append_leaf(BinaryTreeNode("12"), "v_{21.5}","right")

In [ ]:
# Eine _wirklich_ nützliche Hilfsfunktion für das Zeichnen, die mit
# depth-first-traversal aufgerufen werden kann: Notiere einzelne
# Levels, innere Knoten _und_ die Blätter eines binary tree in einem
# Dictionary "account":
def get_list_of_nodes(node,level,account):
    """ Speichere (getrennt) Liste der inneren Knoten und Blätter sowie
    Liste (aller!) Knoten auf den verschiedenen Levels: wenn mit depth-first-Traversal
    aufgerufen, erscheinen die Blätter "von links nach rechts geordnet"."""
    node.contents["level"] = level
    # print(account)
    if "lists of nodes per level" not in account:
        account["lists of nodes per level"] = dict()
    nodes_per_level = account["lists of nodes per level"]

    if level in nodes_per_level:
        nodes_per_level[level].append(node)
    else:
        nodes_per_level[level] = [node]

    if node.is_leaf():
        if "list of leaves" in account:
            account["list of leaves"].append(node)
        else:
            account["list of leaves"] = [node]
    else:
        if "list of inner nodes" in account:
            account["list of inner nodes"].append(node)
        else:
            account["list of inner nodes"] = [node]


In [ ]:
write_function_snippet(get_list_of_nodes,
    **ONLY_PYTHON
)

In [ ]:
# Zeichne einen binary tree in ein PstricksPicture ps:
def draw_binary_tree(ps, binary_tree, width, height, node_draw_func=None, edge_draw_func=None):
    """Zeichne binären Baum in (width x height)-Rechteck, zeichne Kanten
    mit Funktion edge_draw_func (wenn angegeben, default: ps.line) und
    Knoten mit "privater" Zeichenfunktion oder mit node_draw_func (wenn angegeben,
    default: ps.geopoint)."""
    # Denote "matrix-coordinates" of the nodes, and store the nodes
    # (grouped in levels) in level_dict:
    # print(node_draw_func)
    account = dict()
    binary_tree.df_traverse(get_list_of_nodes, 0, account)

    # Schäle die Informationen heraus:
    leaves = account["list of leaves"]
    # Absteigend geordnete Liste der Levels von Knoten
    levels = sorted(list(account["lists of nodes per level"].keys()), reverse=True)
    # Alle Knoten, nach Levels (absteigend) geordnet:
    all_nodes = sum(
        [
            account["lists of nodes per level"][level]
            for level in levels
        ], []
    )

    # Blätter werden äquidistant in x-Richtung aufgeteilt, Levels
    # äquidistant in y-Richtung.
    xstep = width/len(leaves)
    xpos = xstep*0.5
    ystep = height/len(levels)
    # ymax = -ystep*0.5

    for node in leaves:
        # print(f'{node.key}: {node.contents["xy"]}.')
        node.contents["xy"] = npp(xpos, -node.contents["level"]*ystep)
        xpos+= xstep

    for node in all_nodes:
        side = node.node_type # left or right child; or root
        if side != "root":
            # Determine "real coordinates" of parent:
            parent = node.parent
            # Overwrite "real coordinates", even if they already exist:
            #if not "xy" in parent.contents:
            if (sibling := parent.child[OTHER_SIDE[side]]) is not None:
                # print(f'{node.key}: {node.contents["xy"]}, {sibling.key}: {sibling.contents["xy"]}.')
                parent.contents["xy"] = (node.contents["xy"]+sibling.contents["xy"])*0.5+npp(0,ystep)
            else:
                # print(f'{node.key}: {node.contents["xy"]}.')
                parent.contents["xy"] = node.contents["xy"]+npp(0,ystep)
            # Draw edge to parent:
            if edge_draw_func is not None:
                # Es wurde eine spezielle Zeichenfunktion übergeben:
                edge_draw_func(ps, node.contents["xy"], parent.contents["xy"])
            else:
                # Default
                ps.line(node.contents["xy"], parent.contents["xy"])

        # Draw node:
        if "node draw function" in node.contents:
            # Der Knoten hat eine "private" Zeichenfunktion:
            node.contents["node draw function"](ps,node.contents["xy"],node)
        elif node_draw_func is not None:
            # Es wurde eine spezielle Zeichenfunktion übergeben:
            # print("DRAW")
            node_draw_func(ps,node.contents["xy"],node)
        else:
            # Default:
            ps.geopoint(node.contents["xy"])

In [ ]:
write_function_snippet(draw_binary_tree,
    **ONLY_PYTHON
)

In [ ]:
# Adhoc-Funktion für kd-Tree-Graphik, passend zum hier gebastelten Binärbaum -
# die keys sind hier Strings, die die horizontal/vertikale Zerlegung zeigen:
def adhoc_for_kd_tree(ps, pos, node):
    """Adhoc gebastelte Funktion für das Zeichnen von Knoten
    für die "kd-Tree" Graphik."""
    label_parts = node.key.split("_")
    radius = 0.4 if len(label_parts) < 2 else 0.6
    color = "honeydew" if len(label_parts) < 2 else "peachpuff"
    ps.circle(
        pos,
        radius,
        options=f"linecolor=black,fillstyle=solid,fillcolor={color}"
    )
    if len(label_parts) < 2:
        label = fr"{{\tiny ${node.key}$}}"
    else:
        o,s = label_parts
        ps.rput(pos+npp(-0.1,0), fr"{{\sf\white\Large {o.upper()}}}")
        label = fr"{{\tiny ${s}$}}"
        # print(label)
    ps.rput(pos+npp(-0.1,0), label)

In [ ]:
write_function_snippet(adhoc_for_kd_tree,
    **ONLY_PYTHON
)

In [ ]:
ORTH_SHIFT = 0.1
REL_ARROW_LENGTH = 0.4
def draw_edge_with_arrows(ps, node, parent, options="linewidth=0.02,linecolor=gray"):
    """Zeichne Kante mit "begleitenden" Pfeilen, die die depth-first
    Suche illustrieren sollen."""
    # Das ist mal die Kante:
    ps.line(node, parent)
    # Länge der Kante:
    the_vec = parent-node
    edge_len = np.linalg.norm(the_vec)
    directional_offset = edge_len*(1-REL_ARROW_LENGTH)/2
    # Richtungsvektor und Normalvektor der Länge 1:
    rv = get_normalized(the_vec)
    nv = get_normalized_normal(the_vec)
    l2, l1  = [node + (directional_offset*rv + ORTH_SHIFT*nv), parent + (-directional_offset*rv + ORTH_SHIFT*nv)]
    ps.arrow(l1, l2, options=options)
    r1, r2  = [node + (directional_offset*rv - ORTH_SHIFT*nv), parent + (-directional_offset*rv - ORTH_SHIFT*nv)]
    ps.arrow(r1, r2, options=options)
    

In [ ]:
# Die "echte" Graphik "kd_search2" wurde mit der adhoc gebastelten "Knoten-Zeichen-Funktion" adhoc_for_kd_tree erzeugt:
@ps_dmti
def test_bt_graphics(binary_tree, width, height, node_draw_func=None, edge_draw_func=draw_edge_with_arrows, style=None):
    ps = PstricksPicture()
    draw_binary_tree(ps, binary_tree, width, height, node_draw_func=node_draw_func, edge_draw_func=edge_draw_func)
    return ps
make_standalone_pdf(*test_bt_graphics(kdt, 15, 9, filename="depth_search"), showthis=True)

In [ ]:
EPS = 0.3
@ps_dmti
def bt_graphics(binary_tree, width, height, node_draw_func=None, edge_draw_func=None, style=None):
    ps = PstricksPicture()
    draw_binary_tree(ps, binary_tree, width, height, node_draw_func=node_draw_func, edge_draw_func=edge_draw_func)
    # Erzeuge Liste der Knoten, geordnet nach Level:
    account=dict()
    binary_tree.df_traverse(get_list_of_nodes,0,account)
    nodes_per_level = account['lists of nodes per level']
    print(nodes_per_level.keys())
    for level in list(nodes_per_level.keys())[1:]:
        # Runter auf aktuelles Level:
        A,B = nodes_per_level[level-1][-1].contents["xy"], nodes_per_level[level][0].contents["xy"]
        rv = get_normalized(B-A)
        ps.arrow(A+EPS*rv,B-EPS*rv, options="linewidth=0.02,linecolor=gray")
        # Entlang aktuellem Level:
        curr_level = nodes_per_level[level]
        rv = npp(1,0)
        for X,Y in zip(curr_level[:-1], curr_level[1:]):
            A,B = [P.contents["xy"] for P in [X,Y]]
            ps.arrow(A+EPS*rv,B-EPS*rv, options="linewidth=0.02,linecolor=gray")
    return ps
make_standalone_pdf(*bt_graphics(kdt, 15, 9, filename="breadth_search"), showthis=True)

In [ ]:
# MOODLE

## AVL-trees

AVL-Bäume sind im Wesentlichen binäre Such-Bäume, für die Anfügen und Entfernen von Blättern ein "Rebalancing" erfordert, daher sollten wir für alle Teilbäume (also im Wesentlichen: Knoten) in einem AVL-Baum auch die _Längen_ speichern.

Wenn wir AVL-Bäume als _Sub-Klasse_ von binären Bäumen definieren, dann "erbt" diese Sub-Klasse die member-Variablen und member-Funktion der Super-Klasse (also: binäre Bäume): Wir müssen diese nicht mehr definieren, können sie aber "überschreiben" - konkret passiert dieses "Überschreiben" hier für die member-Funktionen ```__init__``` und ```__str__```.

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
# Kleiner Trick (wie bei DoublyLinkedList): "left" <-> "right".
OTHER_SIDE = {
    "left" : "right",
    "right" : "left"
}

# Eine Sub-Klasse in Python - "erbt" von BinaryTreeNode:
class AVLTreeNode(BinaryTreeNode):
    """Class implementing (nodes of) AVL-trees, as a subclass of
    BinaryTreeNode: The key should be numeric (it is used for sorting).
    Recall: Since an AVLNode is a BinaryTreeNode, it serves also as root
    of the subtree pending from it!"""
    def __init__(self, key, data=None, parent = None, node_type = "root"):
        """Initialize:"""
        # Initialize the parent class (BinaryTreeNode), obtained by super():
        super().__init__(key, None, parent, node_type)
        
        # Hier ist es einfacher, die Daten in einer Liste _im_ Dictionary
        # contents zu speichern:
        if data is None:
            self.contents["data"] = []
        else:
            self.contents["data"] = [data]
        # Additionally, we need the lengths (levels) of left/right subtrees:
        # (Denn diese Information brauchen wir für das "rebalancing".)
        self.own_length = 0
        self.subtree_length = {
            'left': -1,
            'right': -1
        }

    def __str__(self):
        """Return string representation:"""
        # Aufruf der Funktion __str__ der super-Klasse:
        l,r = self.subtree_length["left"], self.subtree_length["right"]
        return super().__str__()+f', (l,r) = {(l,r)}.'

    # Auxiliary functions:
    def update_own_length(self):
        """Compute own length:"""
        # Wenn die Längen der an self hängenden Teilbäumen richtig
        # gespeichert wurden, ist das ganz leicht:
        self.own_length = max(
            self.subtree_length["left"],
            self.subtree_length["right"]
        ) + 1
        return self.own_length

    # Wichtige Hilfsfunktion für die "Rotation": Ersetze den Teilbaum
    # side durch den neuen Teilbaum new_subtree (noch _ohne_ "rebalancing"),
    # aber mit Buchführung über eventuell geänderte Längen von Teilbäumen:
    def replace_subtree(self, new_subtree, side):
        """Replace the side (left or right) subtree by new_subtree,
        return the subtree thus replaced."""
        if new_subtree == None:
            # new_subtree might by None: In this case, there are, of course,
            # no members parent and node_type.
            new_subtree_length = -1
        else:
            new_subtree.parent = self
            new_subtree.node_type = side
            new_subtree_length = new_subtree.own_length 

        # Wurzel des Teilbaums, der ersetzt wird:
        replaced_subtree = self.child[side]

        # Ersetzen, und Buchführen über die Länge des neuen Teilbaums:
        self.child[side] = new_subtree
        self.subtree_length[side] = new_subtree_length
        self.update_own_length()
        
        # Eigentlich müssten wir auch für alle Teilbäume "oberhalb" von
        # self die Längen anpassen - aber die Funktion replace_subtree
        # soll NUR verwendet werden, wenn diese Anpassung gleich danach
        # erfolgt. Die Länge von self ist nun aber jedenfalls richtig,
        # wenn die Länge von new_subtree richtig war.
        
        # Nothing has changed "locally" in the replaced subtree!
        # Diesen "abgehängten" Teilbaum müssen wir in den AVL-Rotationen
        # woanders anhängen!
        return replaced_subtree
    
    def move_up_subtree(self, side):
        """Move up subtree side (assumed to be not None!) and return it
        (this is the "rotation" according to Adelson-Velski and Landis)"""

        # Die andere Seite:
        otherside = OTHER_SIDE[side]
        
        # Store information for later use:
        # (in der Graphik aus der Vorlesung: self ist Knoten r)
        node_parent = self.parent
        node_type = self.node_type
    
        # This subtree should "move up":
        # (in der Graphik aus der Vorlesung: Knoten u)
        up_mover = self.child[side]
        
        # This subtree (might be None!) should "change sides":
        # (in der Graphik aus der Vorlesung: Knoten s)
        side_changer = up_mover.child[otherside]
    
        # Replace subtrees:

        # Wir hängen dreimal Teilbäume an, von denen wir
        # wissen, dass ihre Längen richtig sind:

        # Fügen Sie hier den passenden Code ein:
        
        # Längen von Teilbäumen "oberhalb von u" (die u = up_mover
        # enthalten) sind möglicherweise nicht richtig erfasst:
        # Für das "Rebalancing" (und die richtige Erfassung aller
        # Teilbaum-Längen "oberhalb") müssen wir mit dem Knoten u
        # weitermachen.
        return up_mover

    def rebalance_subtrees(self):
        """Rebalance subtrees: Recall that for _every_ node we want
        to maintain the condition
            (ALL keys in the left subtree)
            <= node.key
            < (ALL keys in the right subtree),
        and balancing means maintaining the condition
            -1
            <= (self.subtree_length["left"] - self.subtree_length["right"])
            <= 1.
        """
        # self.update_own_length()
        # "Bias" of the left subtree:
        left_bias=self.subtree_length['left']-self.subtree_length['right']
        if left_bias < -1:
            side = 'right'
        elif left_bias > 1:
            side = 'left'
        else:
            # Bias in [-1,1]: No rebalancing required:
            return self
    
        biased_subtree = self.child[side]

        otherside = OTHER_SIDE[side]

        # Fügen Sie hier den passenden Code ein (einfache Rotation,
        # eventuell mit Vorbereitung):
        
        
        # Geben Sie den Knoten zurück, bei dem
        # Sie das "Rebalancing" fortsetzen müssen.
        return # Code einfügen!
        
    def rebalance(self):
        """Rebalance all nodes, ascending up to the root:
        """

        # Fügen Sie hier den passenden Code ein: Sie müssen
        # im Baum "nach oben" wandern und immer wieder
        # "rebalancieren"; beachten Sie: Sie müssen die Längen
        # der an self hängenden Teilbäume in self.subtree_length
        # updaten!

        # Return the root (node with no parent), but update its
        # length first:
        # Verwenden Sie update_own_length() für die (möglicherweise
        # geänderte) Wurzel und geben Sie die Wurzel zurück:
        return # Code einfügen

    def binary_search(self, key):
        """Find the position where to append key _as a leaf_:"""
        # Fügen Sie hier den passenden Code ein:

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'AVLTreeNode',
    **COMMON,
    the_caption = r'Knoten in einem AVL--Baum, als Sub--Klasse von \pythoncode{BinaryTreeNode}.',
    preamble = ''
)

In der folgenden Sub-Klasse von ```BinaryTree``` sind nur mehr die member-Funktion ```append``` und ```delete``` zu implementieren:

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
class AVLTree(BinaryTree):
    """Basically, a AVLTree is defined by its root, an AVLTreeNode
    (and all the subtrees pending from it)."""
    def __init__(self, key, data = None):
        """Initialize"""
        self.root = AVLTreeNode(key, data=data)
    
    def __str__(self):
        return f'AVLTree: {self.root})'

    # Beim Anfügen und Entfernen von Blättern müssen wir "rebalancen",
    # wenn nötig: Ausserdem kann es "in der Praxis" vorkommen, dass wir
    # mehrere data-Einträge unter demselben key speichern wollen (darum
    # ist contents.data in AVLTreeNode ja auch als Liste definiert), das
    # implementieren wir hier auch.
    def append(self, key, data=None):
        """Append new data in leaf key."""
    
        # Find leaf node where key should be appended as new leaf:
        node, side = self.root.binary_search(key)
        
        # If this key is _equal_ to the existing node.key, simply _append_
        # the data to node.contents:
        if key == node.key:
            node.contents["data"].append(data)
            # Nichts weiter zu tun!
            return

        
        # Implicit else: key != node.key

        # Append a new leaf, using function replace_subtree:
        node.replace_subtree(AVLTreeNode(key, data), side)
        
        other_side = OTHER_SIDE[side]
        # Former leaf node "slides down" to other side:
        node.replace_subtree(
            AVLTreeNode(node.key, node.contents),
            other_side
        )
        # Former leaf node now is an inner node (containing no data):
        node.contents["data"] = []
        # Update key of node:
        if side == 'left':
            node.key = key
                
        # Rebalancing might change the root of the rebalanced tree:
        self.root = node.rebalance()
            
    def delete(self, key, data=None):
        """Find the leaf with key: If there is such leaf, remove the
        data; _if_ remaining data is empty, then also remove the leaf."""
    
        node, side = self.root.binary_search(key)
        if side == 'left' and node.key == key:
            # Ok: There is a leaf node with this key!
            if data is not None:
                try:
                    node.contents["data"].remove(data)
                    # Sind noch andere Daten in diesem node gespeichert?
                    remove_this_leaf = (node.contents["data"] == [])
                except ValueError:
                    print(f"Node {key} does not contain {data}!")
                    # Hmm: Diese Daten gibt´s gar nicht! Der Baum wird
                    # also nicht verändert, und wir "springen aus der
                    # Funktion heraus".
                    return
            else:
                remove_this_leaf = True

            # Implicit else:
            if remove_this_leaf:
                # Ok, this leaf node actually should be deleted:

                parent = node.parent
                side = node.node_type
                other_side = OTHER_SIDE[side]
                
                # Fügen Sie hier den passenden Code ein: Wenn das
                # Blatt gleich der Wurzel ist, setzen Sie self.root = None,
                # andernfalls entfernen Sie das Blatt und rebalancieren
                # Sie den Baum.

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'AVLTree',
    **COMMON,
    the_caption = r'AVL--Baum, als Sub-Klasse von BinaryTree.',
    preamble = ''
)

In [ ]:
# OMIT - das sollte in binary_trees.py übernommen werden!!!

### Graphik: AVL-Bäume

In [ ]:
CELLNUMBER = len(_ih) - 1 
# Kleiner Trick (genau wie bei DoublyLinkedLists): "left" <-> "right".
OTHER_SIDE = {
    "left" : "right",
    "right" : "left"
}

# Eine Sub-Klasse in Python:
class AVLTreeNode(BinaryTreeNode):
    """Class implementing (nodes of) AVL-trees, as a subclass of BinaryTreeNode: The
    key should be numeric (it is used for sorting).
    Recall: Since an AVLNode is a BinaryTreeNode, it serves also as root of the
    subtree pending from it!"""
    def __init__(self, key, data=None, parent = None, node_type = "root"):
        """Initialize:"""
        # Initialize the parent class (BinaryTreeNode), obtained by super():
        super().__init__(key, None, parent, node_type)
        
        # Hier ist es einfacher, die Daten in einer Liste _im_ Dictionary
        # contents zu speichern:
        if data is None:
            self.contents["data"] = []
        else:
            self.contents["data"] = [data]
        # Additionally, we need the lengths (levels) of left/right subtrees:
        # (Denn diese Information brauchen wir für das "rebalancing".)
        self.own_length = 0
        self.subtree_length = {
            'left': -1,
            'right': -1
        }

    def __str__(self):
        """Return string representation:"""
        # Aufruf der Funktion __str__ der super-Klasse:
        return super().__str__()+f', (l,r) = {self.subtree_length["left"], self.subtree_length["right"]}.'

    # Auxiliary functions:
    
    def update_own_length(self):
        """Compute own length:"""
        # Wenn die Längen der an self hängenden Teilbäumen richtig gespeichert
        # wurden, ist das ganz leicht:
        self.own_length = max(self.subtree_length["left"], self.subtree_length["right"]) + 1
        return self.own_length

    # Wichtige Hilfsfunktion für die "Rotation": Ersetze den Teilbaum
    # side durch den neuen Teilbaum new_subtree (noch _ohne_ "rebalancing"),
    # aber mit Buchführung über eventuell geänderte Längen von Teilbäumen:
    def replace_subtree(self, new_subtree, side):
        """Replace the side (left or right) subtree by new_subtree,
        return the subtree thus replaced."""
        if new_subtree == None:
            # new_subtree might by None: In this case, there are no members
            # parent and node_type.
            new_subtree_length = -1
        else:
            new_subtree.parent = self
            new_subtree.node_type = side
            new_subtree_length = new_subtree.own_length # new_subtree.update_own_length()

        # Wurzel des Teilbaums, der ersetzt wird:
        replaced_subtree = self.child[side]

        # Ersetzen, und Buchführen über die Länge des neuen Teilbaums:
        self.child[side] = new_subtree
        self.subtree_length[side] = new_subtree_length
        self.update_own_length()
        # ?
        # if self.parent:
        #     self.parent.update_own_length()
        
        # Eigentlich müssten wir auch für alle Teilbäume "oberhalb" von
        # self die Längen anpassen - aber die Funktion replace_subtree
        # soll NUR verwendet werden, wenn diese Anpassung gleich danach
        # erfolgt. Die Länge von self ist nun aber jedenfalls richtig,
        # wenn die Länge von new_subtree richtig war.
        
        # Nothing has changed "locally" in the replaced subtree!
        # Diesen "abgehängten" Teilbaum müssen wir in den AVL-Rotationen
        # woanders anhängen!
        return replaced_subtree
    
    def move_up_subtree(self, side):
        """Move up subtree side (assumed to be not None!) and return it
        (this is the "rotation" according to Adelson-Velski and Landis)"""

        # Die andere Seite:
        otherside = OTHER_SIDE[side]
        
        # Store information for later use:
        # (in der Graphik aus der Vorlesung: self ist Knoten r)
        node_parent = self.parent
        node_type = self.node_type
    
        # This subtree should "move up":
        # (in der Graphik aus der Vorlesung: Knoten u)
        up_mover = self.child[side]
        
        # This subtree (might be None!) should "change sides":
        # (in der Graphik aus der Vorlesung: Knoten s)
        side_changer = up_mover.child[otherside]
    
        # Replace subtrees:

        # Wir hängen dreimal Teilbäume an, von denen wir
        # wissen, dass ihre Längen richtig sind:
        
        # 1.:(in der Graphik aus der Vorlesung: Knoten s
        # wird an Knoten r angehängt, auf der Seite von u)
        self.replace_subtree(side_changer, side)
        # Länge von self = r ist nun richtig.
        
        # 2.: (in der Graphik aus der Vorlesung: Knoten r
        # wird an Knoten u angehängt, auf _anderen_ Seite von u)
        up_mover.replace_subtree(self, otherside)
        # Länge von up_mover = u ist nun auch richtig.

        # 3.: (in der Graphik aus der Vorlesung: Knoten u
        # ersetzt Knoten r als "Kind" des "Elternknotens"
        # von r; natürlich nur wenn dieser "Elternknoten"
        # existiert (wenn nicht, ist u die neue Wurzel des Baumes).
        if node_parent is not None:
            node_parent.replace_subtree(up_mover, node_type)
            # Länge von self.parent ist nun auch richtig.
        else:
            up_mover.parent = None
            up_mover.node_type = 'root'

        # Längen von Teilbäumen "oberhalb" (die u = up_mover enthalten) sind
        # möglicherweise nicht richtig:
        # Für das "Rebalancing" (und die richtige Erfassung aller Teilbaum-Längen
        # "oberhalb") müssen wir mit dem Knoten u weitermachen.
        return up_mover

    def rebalance_subtrees(self):
        """Rebalance subtrees:
        Recall that for _every_ node we want to maintain the condition
        (ALL keys in the left subtree) <= node.key < (ALL keys in the right subtree),
        and balancing means maintaining the condition
        -1 <= (self.subtree_length["left"] - self.subtree_length["right"]) <= 1.
        """
        # self.update_own_length()
        # "Bias" of the left subtree:
        left_bias = self.subtree_length['left'] - self.subtree_length['right']
        if left_bias < -1:
            side = 'right'
        elif left_bias > 1:
            side = 'left'
        else:
            # Bias in [-1,1]: No rebalancing required:
            return self
    
        biased_subtree = self.child[side]

        otherside = OTHER_SIDE[side]
        
        # Perform additional "up move" for biased_subtree, if necessary:
        # Das ist die "vorbereitende Rotation", siehe die zweite Graphik
        # aus der Vorlesung.
        if biased_subtree.subtree_length[otherside] > biased_subtree.subtree_length[side]:
            biased_subtree.move_up_subtree(otherside)

        # Für die an self hängenden Teilbäume sind nun die Längen richtig
        # erfasst, und es ist alles für die "abschließende Rotation"
        # vorbereitet:
                
        # In any case, perform "up move" for node:
        # Das ist die "einfache Rotation", siehe die erste Graphik
        # aus der Vorlesung. Wir geben den Knoten zurück, bei dem
        # wir das "Rebalancing" fortsetzen müssen.
        return self.move_up_subtree(side)
        
    def rebalance(self):
        """Rebalance all nodes, ascending up to the root:
        """
        node_to_return = self
        node = self
        while node is not None:
            # Update: Richtige Längen von den child-Knoten!
            for side in ['left', 'right']:
                sub_node = node.child[side]
                node.subtree_length[side] = -1 if sub_node is None else sub_node.own_length
            # Rebalance node, store return value:
            node_to_return = node.rebalance_subtrees()
            node_to_return.update_own_length()
            # Ascend upwards in the tree:
            node = node_to_return.parent
    
        # Return the root (node with no parent), but update its length first:
        node_to_return.update_own_length()
        return node_to_return

    def binary_search(self, key):
        """Find the position where to append key _as a leaf_:"""
        if key <= self.key:
            subtree = self.child['left']
            if subtree == None:
                # Knoten key gehört hier angehängt:
                return self, 'left'
        elif key > self.key:
            subtree = self.child['right']
            if subtree == None:
                # Knoten key gehört hier angehängt:
                return self, 'right'
    
        # Implicit else: Continue recursively!
        return subtree.binary_search(key)
    
    def find_extremum(self, side): # Wozu ???
        """Find extremal key (minimal for 'left', maximal for 'right')
        in subtree rooted at self:"""
        node = self
        while (subtree := node.child[side]): # Walrus-operator!
            node = subtree
        return node.key

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'AVLTreeNode',
    **ONLY_PYTHON,
    the_caption = r'AVL--Knoten, als Sub-Klasse von BinaryTreeNode.',
    preamble = ''
)

In [ ]:
CELLNUMBER = len(_ih) - 1
class AVLTree(BinaryTree):
    """Basically, a AVLTree is defined by its root, an AVLTreeNode
    (and all the subtrees pending from it)."""
    def __init__(self, key, data = None, parent = None, node_type = "root"):
        """Initialize"""
        self.root = AVLTreeNode(key, data=data)
    
    def __str__(self):
        return f'AVLTree: {self.root})'

    # Beim Anfügen und Entfernen von Blättern müssen wir "rebalancen",
    # wenn nötig: Ausserdem kann es "in der Praxis" vorkommen, dass wir
    # mehrere data-Einträge unter demselben key speichern wollen (darum
    # ist data in AVLTreeNode ja auch als Liste definiert), das
    # implementieren wir hier auch.
    def append(self, key, data=None):
        """Append new data in leaf key."""
    
        # Find leaf node where key should be appended as new leaf:
        node, side = self.root.binary_search(key)
        
        # If this key is _equal_ to the existing node.key, simply _append_
        # the data to node.contents:
        if key == node.key:
            node.contents["data"].append(data)
            return
        
        # Implicit else: key != node.key
        
        # Append a new leaf, using function replace_subtree:
        node.replace_subtree(AVLTreeNode(key, data), side)
        
        other_side = OTHER_SIDE[side]
        # Former leaf node "slides down" to other side:
        node.replace_subtree(AVLTreeNode(node.key, node.contents), other_side)
        # Former leaf node now is an inner node (containing no data):
        node.contents["data"] = []
        # Update key of node:
        if side == 'left':
            node.key = key
                
        # Rebalancing might change the root of the rebalanced tree:
        self.root = node.rebalance()
            
    def delete(self, key, data=None):
        """Find the leaf with key: If there is such leaf, remove the data;
        _if_ remaining data is empty, then also remove the leaf."""
    
        node, side = self.root.binary_search(key)
        if side == 'left' and node.key == key:
            # Ok: There is a leaf node with this key!
            if data is not None:
                try:
                    node.contents["data"].remove(data)
                    # Sind noch andere Daten in diesem node gespeichert?
                    remove_this_leaf = (node.contents["data"] == [])
                except ValueError:
                    print(f"Node {key} does not contain {data}!")
                    # Hmm: Diese Daten gibt´s gar nicht! Der Baum wird
                    # also nicht verändert, und wir "springen aus der
                    # Funktion heraus".
                    return
            else:
                remove_this_leaf = True

            # Implicit else:
            if remove_this_leaf:
                # Ok, this leaf node actually should be deleted:
                parent = node.parent
                side = node.node_type
                other_side = OTHER_SIDE[side]
                if parent is not None:
                    # Slide upwards sibling (child on other side):
                    sibling = parent.child[other_side]
                    parent.key = sibling.key
                    parent.contents = sibling.contents
                    # Parent node now became a leaf:
                    for s in [side, other_side]:
                        parent.child[s] = None
                        parent.subtree_length[s] = -1
                    parent.own_length = 0
                
                    self.root = parent.rebalance()
                else:
                    # This leaf node is the root node: If we delete it,
                    # there is nothing left.
                    self.root = None

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'AVLTree',
    **ONLY_PYTHON,
    the_caption = r'AVL--Baum, als Sub-Klasse von BinaryTree.',
    preamble = ''
)

In [ ]:
def bf_traverse_print(node, level, count, account=None):
    """Funktion, die als Argument von breadth-first-Durchlauf aufgerufen
    werden kann."""
    if node.is_leaf():
        print(f'({count},{-level}) LEAF: key = {node.key}, data = {node.contents}.')
    else:
        info = " ".join([f"{node.child[side].key}/{node.child[side].own_length}" for side in ['left','right']])
        print(f'({count},{-level}) key = {node.key}: {info}')

In [ ]:
def key_n_length_in_circle(ps, pos, node):
    """Einfache Funktion, die einen Knoten in einem AVL-Baum zeichnet."""
    ps.circle(pos, 0.45, options="linecolor=white,fillstyle=solid,fillcolor=peachpuff")
    # ps.rput(pos+npp(-0.1,0), fr"{{\tiny ${node.key}$/{{\lightgray ${node.own_length}$}}}}")
    if node.is_leaf():
        ps.rput(pos+npp(-0.1,0), fr"{{\tiny ${node.key}$}}")
    else:
        ps.rput(pos+npp(0.8,0), fr"{{\small\lightgray ?}}")
        ps.rput(pos+npp(-0.1,0), fr"{{\tiny\gray $\leq\! {node.key}$}}")

In [ ]:
@ps_dmti
def show_evolving_AVL_tree(to_insert, width, height, node_draw_func=key_n_length_in_circle, label="", style=None):
    ps = PstricksPicture()

    ps.rput(npp(width/2, 0.8), r"{\tiny\gray Ein AVL--Baum \dots}")
    AVL = AVLTree(to_insert[0])
    for key in to_insert[1:]:
        AVL.append(key)
    
    draw_binary_tree(ps, AVL, width, height, node_draw_func=node_draw_func) # simple_key_in_circle
    
    if label:
        ps.rput(npp(width/2,-height+0.2), label)
        ps.extend(npp(0,0.4),npp(0,0))
    
    ps.extend(npp(0,0),npp(0,0.4))
    return ps

# make_standalone_pdf(*show_evolving_AVL_tree(lok, 12, 8, label=r"{\small\gray Baum}", node_draw_func=key_n_length_in_circle), showthis=True)

In [ ]:
def key_n_length_in_circle(ps, pos, node):
    """Einfache Funktion, die einen Knoten in einem AVL-Baum zeichnet."""
    ps.circle(pos, 0.45, options="linecolor=white,fillstyle=solid,fillcolor=peachpuff")
    # ps.rput(pos+npp(-0.1,0), fr"{{\tiny ${node.key}$/{{\lightgray ${node.own_length}$}}}}")
    if node.is_leaf():
        ps.rput(pos+npp(-0.1,0), fr"{{\tiny ${node.key}$}}")
    else:
        ps.rput(pos+npp(0.55,0), fr"{{\small\lightgray ?}}")
        ps.rput(pos+npp(-0.1,0), fr"{{\tiny\gray $\leq\! {node.key}$}}")

# List of keys:
lok = [0,4,5,6,3,2,1, 42, 1, -2]
for nok in range(2, len(lok)):
    if nok == 2:
        label = r"{\tiny\gray \dots\ \anf{im Entstehen}.}"
    else:
        label = rf"{{\tiny\gray \dots\ nach Anhängen von Blatt {lok[nok-1]}.}}"
    # show_evolving_AVL_tree(lok[:nok], 8, 5, node_draw_func=key_n_length_in_circle, filename=f"AVL-{nok}", label=label, style=None)


In [ ]:
# MOODLE